In [ ]:
#lesson19 網路爬蟲Web Crawler基本教學
#抓取網頁原始碼(HTML)
import urllib.request as req
url="https://www.ptt.cc/bbs/movie/index9503.html"
#建立一個 Request 物件，附加 Request Headers 的資訊，需要headers才看起來像一般使用者
request=req.Request(url, headers={
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36)"
})
with req.urlopen(request) as response:
    data=response.read().decode("utf-8")

#解析原始碼，取得看板的title
#直接在下面 TERMINAL 安裝 pip install BeautifulSoup4
import bs4
root=bs4.BeautifulSoup(data, "html.parser") #把剛抓下來的data，讓BeautifulSoup協助我們解析HTML格式文件
print(root.title.string)

#解析原始碼，取得每篇文章的標題
import bs4
root=bs4.BeautifulSoup(data, "html.parser") #把剛抓下來的data，讓BeautifulSoup協助我們解析HTML格式文件
titles=root.find("div", class_="title") #尋找class="title"的div標籤(不確定為啥要加一個橫槓在class後面，但不加好像不能跑)
print(titles.a.string)

#上面的root.find只找出一項，所以要用root.find_all
import bs4
root=bs4.BeautifulSoup(data, "html.parser") #讓 BeautifulSoup 協助我們解析 HTML 格式文件
titles=root.find_all("div", class_="title") #尋找class="title"的div標籤
for title in titles:
    if title.a != None: #如果標題包含a標籤(該文章沒有被刪除)，印出來
        print(title.a.string)

In [ ]:
#lesson20 網路爬蟲Web Crawler教學-Cookie操作實務
#抓取網頁原始碼(HTML)
import urllib.request as req
def getData(url):
    request=req.Request(url, headers={
        #跟lesson 19 的不同在於cookie，因為八卦版進入前會先問是否已滿十八歲，我們要先閃過他，記得加他媽的逗號
        "cookie":"over18=1",
        "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data=response.read().decode("utf-8")

    import bs4
    root=bs4.BeautifulSoup(data, "html.parser")
    titles=root.find_all("div", class_="title") #尋找class="title"的div標籤
    for title in titles:
        if title.a != None: #如果標題包含a標籤(該文章沒有被刪除)，印出來
            print(title.a.string)
    #抓取上一頁面的連結
    nextlink=root.find("a", string="‹ 上頁") #找到內文是‹ 上頁 的a標籤
    return nextlink["href"]

#主程序: 抓取多個頁面的標題
pageURL="https://www.ptt.cc/bbs/Gossiping/index.html"
count=0
#做了一個迴圈想抓幾頁自己安排
while count<3:
    pageURL="https://www.ptt.cc"+getData(pageURL)
    count+=1

In [ ]:
#lesson21 網路爬蟲Web Crawler教學-AJAX/XHR網站技術分析實務
#由於新版的Medium跟老師上的版本已經玩群不同，先把老師的程式碼抄下來，但沒跑過
#抓取Medium.com的文章資料
import urllib.request as req
url="https://medium.com/"
request=req.Request(url, headers={
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
})
with req.urlopen(request) as response:
    data=response.read().decode("utf-8") #根據觀察，取得的資料是JSON的格式
    
#解析JSON格式的資料，取得每篇文章的標題
import json
data=data.replace("])}while(1);</x>","") #先將莫名其妙的字串轉換為空字串
data=json.loads(data) #把原始的JSON資料解析成字典/列表的表示形式
#取得JSON資料中的文章標題
posts=data["payload"]["references"]["Post"]
for key in posts:
    post=posts[key]
    print(post["title"])

In [28]:
#lesson22 網路爬蟲Web Crawler教學-Request Data操作實務
#抓取Medium.com的文章資料新版，加入Request Data觀念
import urllib.request as req
import json
#建立連線網址
url="https://medium.com/_/graphql"
#建立一個Request物件，附上Request Headers和Request Data的資訊(總共三個參數)
#將requestData內容從字典改成字串
requestData=[{"operationName":"TopicHandlerHomeFeed","variables":{"topicSlug":"editors-picks","feedPagingOptions":{"limit":25,"to":"1652213118384"}},"query":"query TopicHandlerHomeFeed($topicSlug: ID!, $feedPagingOptions: PagingOptions) {\n  topic(slug: $topicSlug) {\n    ...CuratedHomeFeedItems_topic\n    __typename\n  }\n}\n\nfragment CuratedHomeFeedItems_topic on Topic {\n  id\n  name\n  latestPosts(paging: $feedPagingOptions) {\n    postPreviews {\n      postId\n      post {\n        id\n        ...HomeFeedItem_post\n        __typename\n      }\n      __typename\n    }\n    pagingInfo {\n      next {\n        limit\n        to\n        __typename\n      }\n      __typename\n    }\n    __typename\n  }\n  __typename\n}\n\nfragment HomeFeedItem_post on Post {\n  __typename\n  id\n  title\n  firstPublishedAt\n  mediumUrl\n  collection {\n    id\n    name\n    domain\n    logo {\n      id\n      __typename\n    }\n    __typename\n  }\n  creator {\n    id\n    name\n    username\n    imageId\n    mediumMemberAt\n    __typename\n  }\n  previewImage {\n    id\n    __typename\n  }\n  previewContent {\n    subtitle\n    __typename\n  }\n  readingTime\n  tags {\n    ...TopicPill_tag\n    __typename\n  }\n  ...BookmarkButton_post\n  ...OverflowMenuButtonWithNegativeSignal_post\n  ...PostPresentationTracker_post\n  ...PostPreviewAvatar_post\n}\n\nfragment TopicPill_tag on Tag {\n  __typename\n  id\n  displayTitle\n}\n\nfragment BookmarkButton_post on Post {\n  visibility\n  ...SusiClickable_post\n  ...AddToCatalogBookmarkButton_post\n  __typename\n  id\n}\n\nfragment SusiClickable_post on Post {\n  id\n  mediumUrl\n  ...SusiContainer_post\n  __typename\n}\n\nfragment SusiContainer_post on Post {\n  id\n  __typename\n}\n\nfragment AddToCatalogBookmarkButton_post on Post {\n  ...AddToCatalogBase_post\n  __typename\n  id\n}\n\nfragment AddToCatalogBase_post on Post {\n  id\n  __typename\n}\n\nfragment OverflowMenuButtonWithNegativeSignal_post on Post {\n  id\n  ...OverflowMenuWithNegativeSignal_post\n  ...CreatorActionOverflowPopover_post\n  __typename\n}\n\nfragment OverflowMenuWithNegativeSignal_post on Post {\n  id\n  creator {\n    id\n    __typename\n  }\n  collection {\n    id\n    __typename\n  }\n  ...OverflowMenuItemUndoClaps_post\n  __typename\n}\n\nfragment OverflowMenuItemUndoClaps_post on Post {\n  id\n  clapCount\n  ...ClapMutation_post\n  __typename\n}\n\nfragment ClapMutation_post on Post {\n  __typename\n  id\n  clapCount\n  ...MultiVoteCount_post\n}\n\nfragment MultiVoteCount_post on Post {\n  id\n  ...PostVotersNetwork_post\n  __typename\n}\n\nfragment PostVotersNetwork_post on Post {\n  id\n  voterCount\n  recommenders {\n    name\n    __typename\n  }\n  __typename\n}\n\nfragment CreatorActionOverflowPopover_post on Post {\n  allowResponses\n  id\n  statusForCollection\n  isLocked\n  isPublished\n  clapCount\n  mediumUrl\n  pinnedAt\n  pinnedByCreatorAt\n  curationEligibleAt\n  mediumUrl\n  responseDistribution\n  visibility\n  inResponseToPostResult {\n    __typename\n  }\n  inResponseToCatalogResult {\n    __typename\n  }\n  pendingCollection {\n    id\n    name\n    creator {\n      id\n      __typename\n    }\n    avatar {\n      id\n      __typename\n    }\n    domain\n    slug\n    __typename\n  }\n  creator {\n    id\n    ...MutePopoverOptions_creator\n    ...auroraHooks_publisher\n    __typename\n  }\n  collection {\n    id\n    name\n    creator {\n      id\n      __typename\n    }\n    avatar {\n      id\n      __typename\n    }\n    domain\n    slug\n    ...MutePopoverOptions_collection\n    ...auroraHooks_publisher\n    __typename\n  }\n  ...useIsPinnedInContext_post\n  ...NewsletterV3EmailToSubscribersMenuItem_post\n  ...OverflowMenuItemUndoClaps_post\n  __typename\n}\n\nfragment MutePopoverOptions_creator on User {\n  id\n  __typename\n}\n\nfragment auroraHooks_publisher on Publisher {\n  __typename\n  ... on Collection {\n    isAuroraEligible\n    isAuroraVisible\n    viewerEdge {\n      id\n      isEditor\n      __typename\n    }\n    __typename\n    id\n  }\n  ... on User {\n    isAuroraVisible\n    __typename\n    id\n  }\n}\n\nfragment MutePopoverOptions_collection on Collection {\n  id\n  __typename\n}\n\nfragment useIsPinnedInContext_post on Post {\n  id\n  collection {\n    id\n    __typename\n  }\n  pendingCollection {\n    id\n    __typename\n  }\n  pinnedAt\n  pinnedByCreatorAt\n  __typename\n}\n\nfragment NewsletterV3EmailToSubscribersMenuItem_post on Post {\n  id\n  creator {\n    id\n    newsletterV3 {\n      id\n      subscribersCount\n      __typename\n    }\n    __typename\n  }\n  isNewsletter\n  isAuthorNewsletter\n  __typename\n}\n\nfragment PostPresentationTracker_post on Post {\n  id\n  visibility\n  previewContent {\n    isFullContent\n    __typename\n  }\n  collection {\n    id\n    slug\n    __typename\n  }\n  __typename\n}\n\nfragment PostPreviewAvatar_post on Post {\n  __typename\n  id\n  collection {\n    id\n    name\n    ...CollectionAvatar_collection\n    __typename\n  }\n  creator {\n    id\n    username\n    name\n    ...UserAvatar_user\n    ...userUrl_user\n    __typename\n  }\n}\n\nfragment CollectionAvatar_collection on Collection {\n  name\n  avatar {\n    id\n    __typename\n  }\n  ...collectionUrl_collection\n  __typename\n  id\n}\n\nfragment collectionUrl_collection on Collection {\n  id\n  domain\n  slug\n  __typename\n}\n\nfragment UserAvatar_user on User {\n  __typename\n  id\n  imageId\n  mediumMemberAt\n  name\n  username\n  ...userUrl_user\n}\n\nfragment userUrl_user on User {\n  __typename\n  id\n  customDomainState {\n    live {\n      domain\n      __typename\n    }\n    __typename\n  }\n  hasSubdomain\n  username\n}\n"}]
request=req.Request(url, headers={
    "content-type":"application/json",
    "user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36"
}, data=json.dumps(requestData).encode("utf-8"))
#發出請求
with req.urlopen(request) as response:
    result=response.read().decode("utf-8")

#解析JSON格式的資料，取得每篇文章的標題
result=json.loads(result)
#print(result[0]["data"]["topic"]["latestPosts"]["postPreviews"][0]["post"]["title"]) #試著印出第一篇文章的標題
#注意 items此時為一個列表
items=result[0]["data"]["topic"]["latestPosts"]["postPreviews"]
for item in items:
    print(item["post"]["title"])

Escape Fantasies of the Tech Billionaires
There’s Nothing New About Undermining Women’s Autonomy
How to Make a Mother
Have More Sex, Sleep Expert Advises
How Are You So Easy to Find on the Internet?
Can Linkin Park Ever Replace Chester Bennington?
Why Write?
How Lying Affects Linguistic Expression
Other People’s Anger
Is Allergy Season Getting Worse Every Year?
About those kill-switched Ukrainian tractors
The Return of 1980s-Era Nuclear-Strike Maps
How to Grow Your Child’s Social Skills
The College Kids Are Back to Normal
What About the “Woke” Right?
Why the “Improve by 1% Every Day” Mantra Is Bullsh*t
It’s Not The Size Of Your Sabbatical
Getting Inside Putin’s Head
My Mom Was for the Birds
What Does the Perfect Work Day Look Like?
The Attention Economy is Making Us Stupid
The Abercrombie & Fitch Documentary Reveals How Power Decides What’s Cool
Pregnancy is Dangerous, Debilitating, and Costly Unpaid Work
Men Cause 100% of Unwanted Pregnancies
As NFT Sales Continue to Plummet, Is the B

In [26]:
#lesson23 Flask網站開發-基礎環境建置教學
#pip install flask
from flask import Flask
app=Flask(__name__) #__name__為python內建的變數，他會儲存這個程式在哪個模組下執行(代表目前執行的模組)

@app.route("/") #函式的裝飾(Decorator):以函式為基礎，提供附加的功能
def home():
    return "Hello Flask"

@app.route("/test") #代表我們要處理的網站路徑
def test():
    return "Lannisters send their regards"

if __name__=="__main__": #如果以主程式執行
    app.run() #立刻啟動伺服器

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/May/2022 00:36:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:36:13] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:39:09] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:39:10] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:39:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:39:20] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:39:31] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:40:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/May/2022 00:41:17] "GET /test HTTP/1.1" 200 -


In [1]:
#lesson24 Flask網站開發-Heroku雲端主機教學
#pip install flask
#上網安裝git
#上網安裝heroku
#git
#heroku
#heroku login
#git init
#heroku git:remote -a python-training-huangpohsun
#以上動作為 初始化專案
#git add .
#git commit -m "First Deploy"
#git push heroku master

#若更改網站內容，做以下動作
#git add .
#git commit -m "Second Deploy"
#git push heroku master

from flask import Flask
app=Flask(__name__) #__name__為python內建的變數，他會儲存這個程式在哪個模組下執行(代表目前執行的模組)

@app.route("/") #函式的裝飾(Decorator):以函式為基礎，提供附加的功能
def home():
    return "Hello Flask"

@app.route("/test") #代表我們要處理的網站路徑
def test():
    return "Lannisters send their regards"

if __name__=="__main__": #如果以主程式執行
    app.run() #立刻啟動伺服器


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [16/May/2022 22:19:46] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:19:46] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [16/May/2022 22:19:52] "GET /test HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:21:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:25:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2022 22:25:33] "GET / HTTP/1.1" 200 -
